In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import time
from pprint import pprint
import requests
from datetime import date, timedelta, datetime
import json
from pprint import pprint
from tqdm import tqdm
from tqdm import tqdm_notebook
# sqlite Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

In [ ]:
from nba_api.stats.endpoints import playercareerstats, drafthistory, commonplayerinfo, playerawards

In [ ]:
# Not used
# cumestatsplayer,draftcombinedrillresults,playerdashboardbyteamperformance, leagueleaders, leaguedashplayerstats, draftcombinenonstationaryshooting, draftcombinestats, commonallplayers,

In [ ]:
headers = {'Host': 'stats.nba.com', 
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
           'Accept': 'application/json, text/plain, */*',
           'Accept-Language': 'en-US,en;q=0.5',
           'Accept-Encoding': 'gzip, deflate, br',
           'x-nba-stats-origin': 'stats',
           'x-nba-stats-token': 'true',
           'Connection': 'keep-alive',
           'Referer': 'https://stats.nba.com/',
           'Pragma': 'no-cache',
           'Cache-Control': 'no-cache'}

In [ ]:
# pd.show_versions()

## Static Examples

In [ ]:
from nba_api.stats.static import players
# Find players by full name.
players.find_players_by_full_name('Stromile Swift')

# # Find players by first name.
# players.find_players_by_first_name('lebron')

# # Find players by last name.
# players.find_players_by_last_name('^(james|love)$')

# Get all players.
# players.get_players()

## Begin Database Connection (sqlite)

In [ ]:
# Base = declarative_base()
# engine = create_engine('sqlite:///db.sqlite', echo=False)
# conn = engine.connect()
# Create (if not already in existence) the tables associated with our classes.
# Base.metadata.create_all(engine)
# # Create a Session Object to Connect to DB
# # ----------------------------------
# session = Session(bind=engine)

In [ ]:
# Use this to clear out the db
# ----------------------------------
# Session.rollback(self)
# Base.metadata.drop_all(engine)
# session.commit()

## List of all players
find specific player

In [ ]:
## NEED
from nba_api.stats.static import players
# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
print('Number of players fetched: {}'.format(len(nba_players)))
nba_players[:10]
all_players = pd.DataFrame(nba_players)
all_players = all_players.copy()
all_players

In [ ]:
all_players_clean = all_players.rename(columns={"id": "PLAYER_ID", 'full_name': 'PLAYER_NAME'})
all_players_clean = all_players_clean.drop(columns=['first_name','last_name','is_active'])
all_players_clean

### Find specific player

In [ ]:
## loop to find player
player_name = [player for player in nba_players
                   if player['full_name'] == 'Tim Duncan'][0]
player_name

## Get Career Stats by player id number
Get player_id number from get_players above

#### Key: 

* GP: Games Played
* MIN: Minutes Played
* FGM: Field Goals Made
* FGA: Field Goals Attempted
* FG%: Field Goal Percentage
* 3PM: 3 Point Field Goals Made
* 3PA: 3 Point Field Goals Attempted
* 3P%: 3 Point Field Goals Percentage
* FTM: Free Throws Made
* FTA: Free Throws Attempted
* FT%: Free Throw Percentage
* OREB: Offensive Rebounds
* DREB: Defensive Rebounds
* REB: Rebounds
* AST: Assists
* TOV: Turnovers
* STL: Steals
* BLK: Blocks
* PF: Personal Fouls
* DD2: Double Doubles
* TD3: Trible Doubles
* PTS: Points
* YIL: Year in League




example

In [ ]:
# Display all columns
pd.set_option('display.max_columns', 500)
# Anthony Davis
career = playercareerstats.PlayerCareerStats(player_id='76001')
career_df = career.get_data_frames()[0]

career_dict = career_df.to_dict('records')
first_5_years = career_df[0:5]
first_5_years

In [ ]:
# main_df = pd.DataFrame()
# yil = 1

# def addYil(first_5_years, yil):
#     ## List all season_id column values found
#     season_ids = list(first_5_years['SEASON_ID'].values)
    
#     ## Create list to add to dataframe
#     ## Scans values in SEASON_ID. If change in year is found, adds 1 int to new column 'YIL' (Years in League). 
#     ## We are scanning for 5, 8, and 10 year milestones.
#     ## Changes are scanned due to players under milestones. New rows will add duplicate SEASON_ID if the player changed teams during the season.
#     previous = ''
#     new_yills_add = []
#     for i, value in enumerate(season_ids):
#         print(type(value))
        
#         if value != previous:
#             new_yills_add.append(yil)
#             yil+=1
#     #         print(i)        
#         else:
#             new_yills_add.append(yil - 1)
#             continue

#         previous = season_ids[i]
#     first_5_years_yil = first_5_years.assign(YIL=new_yills_add)  

#     return first_5_years_yil
# first_5_years_yil = addYil(first_5_years, yil)


# main_df = main_df.append( first_5_years_yil, ignore_index = True)


In [ ]:
# ## List all season_id column values found
# season_ids = list(first_5_years['SEASON_ID'].values)

# ## Create list to add to dataframe
# ## Scans values in SEASON_ID. If change in year is found, adds 1 int to new column 'YIL' (Years in League). 
# ## We are scanning for 5, 8, and 10 year milestones.
# ## Changes are scanned due to players under milestones. New rows will add duplicate SEASON_ID if the player changed teams during the season.

# yil = 1
# new_yills_add = []
# previous = ''
# for i, value in enumerate(season_ids):
    
#     if value != previous:
#         new_yills_add.append(yil)
#         yil+=1
# #         print(i)        
#     else:
#         new_yills_add.append(yil - 1)
#         continue
    
#     previous = season_ids[i]
    
# # New dataframe with YIL column
# first_5_years_yil = first_5_years.assign(YIL=new_yills_add)  
# first_5_years_yil

In [ ]:
career_df.columns

-----------

#### PySpark databases 
testing

In [ ]:
# import os
# import findspark
# findspark.init()

In [ ]:
# # Start Spark session
# from pyspark.sql import SparkSession
# from pyspark import SparkFiles
# spark = SparkSession.builder.appName("nbaStats").getOrCreate()

In [ ]:
# with open("sample.json", "w") as outfile:  
#     json.dump(career_dict, outfile) 

In [ ]:
# sc = spark.sparkContext
# path = "./sample.json"
# players_json_spark = spark.read.json(path)

In [ ]:
# players_json_spark.printSchema()

In [ ]:
# Creates a temporary view using the DataFrame
# players_json_spark.createOrReplaceTempView("people")

In [ ]:
# players_json_spark.show()

In [ ]:
# # playersDF_spark = spark.range(3).toDF("myCol")
# newRow = players_json_spark
# appended = players_json_spark.union(newRow)
# display(appended)

In [ ]:
# appended.count()

In [ ]:
# def customFunction(row):

#    return (row)

# sample2 = appended.rdd.map(customFunction)
# sample2

In [ ]:
# for ids in all_players_ids: 
#     player_to_pass = ids

In [ ]:
# for f in appended.collect(): 
#         print (f)

In [ ]:
# playersDF_spark.withColumn('newprice', dataframe]).show()

In [ ]:
# otherPeople = spark.read.json(path)
# otherPeople.show()
# otherPeople.select(otherPeople["AST"]).show()

In [ ]:
# Create DataFrame manually
# dataframe = spark.createDataFrame(path, schema=main_df_columns)
# dataframe.show()

------------

## Get first 5 years of each player id.

In [ ]:
all_players_ids = list(all_players['id'].values.astype(str))
len(all_players_ids)

In [ ]:
all_players_col = ['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'YIL']
len(all_players_col)

In [ ]:
pbar = tqdm(total=len(all_players_ids))
main_df = pd.DataFrame()
yil = 1

def addYil(first_5_years, yil):
    ## List all season_id column values found
    season_ids = list(first_5_years['SEASON_ID'].values)
    
    ## Create list to add to dataframe
    ## Scans values in SEASON_ID. If change in year is found, adds 1 int to new column 'YIL' (Years in League). 
    ## We are scanning for 5, 8, and 10 year milestones.
    ## Changes are scanned due to players under milestones. New rows will add duplicate SEASON_ID if the player changed teams during the season.
    previous = ''
    new_yills_add = []
    
    for i, value in enumerate(season_ids):
#         print(type(value))
        if value != previous:
            new_yills_add.append(yil)
            yil+=1       
        else:
            new_yills_add.append(yil - 1)
            continue

        previous = season_ids[i]
        
    first_5_years_yil = first_5_years.assign(YIL=new_yills_add)  
    return first_5_years_yil


for ids in all_players_ids: 
    player_to_pass = ids

    career = playercareerstats.PlayerCareerStats(player_id=player_to_pass, headers=headers, timeout=100)
    career_df = career.get_data_frames()[0]
    career_df['YIL'] = 0
    first_5_years = career_df[0:5]
    first_5_years_yil = addYil(first_5_years, yil)

    main_df = main_df.append( first_5_years_yil, ignore_index = True)
    time.sleep(1)
    pbar.update(1)
    
pbar.close()


In [ ]:
main_df_pass = main_df.copy()

In [ ]:
#  Player positions
player_positions = pd.read_csv('../../datasets/nba_positions.csv') 
player_positions

In [ ]:
five_year_all_players = main_df_pass.sort_values(by='SEASON_ID', ascending=False).copy()
five_year_all_players = five_year_all_players.reset_index(drop=True)
five_year_all_players = five_year_all_players.join(all_players_clean.set_index('PLAYER_ID'), on='PLAYER_ID')
# five_year_all_players = players_with_names.join(player_positions.set_index('PLAYER_NAME'), on='PLAYER_NAME')
five_year_all_players

In [ ]:
five_year_all_players_to_json = five_year_all_players.to_json(orient='records')

In [ ]:
# SAVE: Player_position
with open(f'./_players_all_data.json', 'w') as fp:
    json.dump(five_year_all_players_to_json, fp)

-------------

# Open Saved JSON

In [ ]:
with open('./_players_all_data.json') as json_file:
    data = json.load(json_file)

In [ ]:
#  Player positions
player_positions = pd.read_csv('../../datasets/nba_positions_no_team.csv') 
player_positions = player_positions.drop_duplicates( keep='first').reset_index(drop=True)
player_positions

In [ ]:
five_year_all_players = pd.read_json(data)
twenty_years_all_players = five_year_all_players[five_year_all_players['SEASON_ID'] > '2000-99']
twenty_years_all_players

#### Merge with position


In [ ]:
twenty_years_all_players = twenty_years_all_players.join(player_positions.set_index('PLAYER_NAME'), on='PLAYER_NAME')
twenty_years_all_players = twenty_years_all_players.dropna()
twenty_years_all_players

In [ ]:
import plotly.express as px


fig = px.scatter(twenty_years_all_players, x="YIL", y="PTS", color="PTS", log_x=True, hover_name="PLAYER_NAME", hover_data=["PTS"])

fig.show()

In [ ]:
# five_year_all_players.set_index('SEASON_ID')

In [ ]:
## Save all players data to sqlite db 'all_players'
# all_players.to_sql('all_players', con=engine)

## Retreive all players from sqlite db
# engine.execute("SELECT * FROM all_players").fetchall()

https://plotly.com/python/polar-chart/

In [ ]:
import plotly.express as px
df = px.data.wind()
fig = px.line_polar(twenty_years_all_players, r="PTS", theta="POSITION", color="YIL", line_close=True,
                    color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark",)
fig.show()

In [ ]:
df = px.data.wind()
df